### Pregunta 1

El algoritmo KNN se basa en encontrar los k valores más cercanos al objetivo seleccionado y mediante una operación como el promedio, obtener una estimación del valor del objetivo. Las métricas de similitud nos permiten conocer cuales son los valores mas similares o cercanos del objetivo.

#### Ventajas

- Fácil de implementar.
- Sirve con datos escasos.

#### Desventajas

- No es muy escalable.
- Se puede ver afectado por una alta cantidad de características (maldición de la dimensionalidad).

### Pregunta 2



In [1]:
# Descargar el dataset MovieLens 1M
!curl -o dataset.zip "https://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip dataset.zip
!ls -la

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5778k  100 5778k    0     0  11.6M      0 --:--:-- --:--:-- --:--:-- 11.6M
Archive:  dataset.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         
total 5800
drwxr-xr-x 1 root root    4096 Oct 29 00:39 .
drwxr-xr-x 1 root root    4096 Oct 29 00:38 ..
drwxr-xr-x 4 root root    4096 Oct 25 13:20 .config
-rw-r--r-- 1 root root 5917549 Oct 29 00:39 dataset.zip
drwxr-x--- 2 root root    4096 Jan 29  2016 ml-1m
drwxr-xr-x 1 root root    4096 Oct 25 13:20 sample_data


In [2]:
# Importar librerias
import pandas as pd

In [3]:
# Cargar los datasets de calificaciones y películas
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, engine='python',
                      names=['userId', 'movieId', 'rating', 'timestamp'], encoding='latin-1')
movies  = pd.read_csv('ml-1m/movies.dat', sep='::',  header=None, engine='python',
                      names=['movieId', 'title', 'genres'], encoding='latin-1')

In [4]:
# Unir los datasets en base a 'movieId' para agregar los títulos
user_item_rating = pd.merge(ratings, movies[['movieId', 'title']], on='movieId')

# Seleccionar solo las columnas necesarias
user_item_rating = user_item_rating[['userId', 'title', 'rating']]

# Ordenar los datos por 'userId'
user_item_rating.sort_values(by='userId', inplace=True)

# Mostrar las primeras filas para verificar
user_item_rating.head()

,userId,title,rating
0,1,One Flew Over the Cuckoo's Nest (1975),5
29,1,"Close Shave, A (1995)",3
30,1,Antz (1998),4
31,1,"Girl, Interrupted (1999)",4
32,1,Hercules (1997),4


In [7]:
# Establecer el mínimo de calificaciones por película
min_user_ratings_per_movie = 5

# Filtrar las películas con al menos min_ratings_per_movie calificaciones
user_rating_counts = user_item_rating['userId'].value_counts()
user_ratings_filtered = user_rating_counts[user_rating_counts >= min_user_ratings_per_movie].index

# Obtener usuarios random filtrados
random_user_1 = user_item_rating[user_item_rating.userId == user_ratings_filtered[0]]
random_user_2 = user_item_rating[user_item_rating.userId == user_ratings_filtered[1]]

random_user_1, random_user_2

(        userId                                             title  rating
 697182    4169                                 After Life (1998)       3
 697184    4169                              Sliding Doors (1998)       4
 697194    4169                           Frogs for Snakes (1998)       2
 697185    4169  Star Wars: Episode I - The Phantom Menace (1999)       4
 697183    4169                       Minnie and Moskowitz (1971)       3
 ...        ...                                               ...     ...
 696411    4169                             Wrong Man, The (1956)       4
 696412    4169                               Fathers' Day (1997)       3
 696413    4169                 Man Who Knew Too Much, The (1956)       3
 696414    4169                         Fifth Element, The (1997)       4
 696415    4169                    Trouble with Harry, The (1955)       4
 
 [2314 rows x 3 columns],
         userId                      title  rating
 279626    1680              Fall

In [8]:
# Identificar las películas comunes
common_movies = pd.merge(random_user_1, random_user_2, on="title")

# Extraer las calificaciones de los usuarios para las películas comunes
user1_ratings = common_movies['rating_x']
user2_ratings = common_movies['rating_y']

# Calcular la correlación de Pearson
pearson_correlation = user1_ratings.corr(user2_ratings)
print("Correlación de Pearson entre los dos usuarios:", pearson_correlation)

Correlación de Pearson entre los dos usuarios: 0.23844186702997186


Este resultado indica una relación positiva débil entre sus calificaciones de películas. En términos de un sistema de recomendación:
- La correlación de 0.238 sugiere que los usuarios tienen algunas coincidencias en sus gustos, pero no son suficientemente fuertes como para que se considere que tienen preferencias claramente alineadas.
- En un sistema de recomendación, esta baja correlación indica que las películas preferidas por uno de los usuarios probablemente no serán altamente relevantes para el otro. Si el sistema basa las recomendaciones principalmente en este nivel de correlación, es posible que no siempre brinde sugerencias precisas o satisfactorias para ambos.

Para una recomendación efectiva, se suele buscar una correlación más alta, idealmente cercana a 1.

### Pregunta 3

In [11]:
# Establecer el mínimo de calificaciones por película
min_ratings_per_movie = 50

# Filtrar las películas con al menos min_ratings_per_movie calificaciones
movie_counts = user_item_rating['title'].value_counts()
popular_movies = movie_counts[movie_counts >= min_ratings_per_movie].index

# Filtrar el dataset original
filtered_data = user_item_rating[user_item_rating['title'].isin(popular_movies)]
filtered_data.head()

,userId,title,rating
0,1,One Flew Over the Cuckoo's Nest (1975),5
29,1,"Close Shave, A (1995)",3
30,1,Antz (1998),4
31,1,"Girl, Interrupted (1999)",4
32,1,Hercules (1997),4


In [15]:
from sklearn.metrics.pairwise import cosine_similarity

# Crear la matriz usuario-ítem
user_item_matrix = filtered_data.pivot_table(index='userId', columns='title', values='rating').fillna(0)

# Seleccionar una película objetivo y dos películas para comparar
movie_target = filtered_data.iloc[0]['title']
movie_compare_1 = filtered_data.iloc[1]['title']
movie_compare_2 = filtered_data.iloc[2]['title']

# Obtener los vectores de calificación de las películas seleccionadas
target_vector = user_item_matrix[movie_target].values.reshape(1, -1)
compare_vector_1 = user_item_matrix[movie_compare_1].values.reshape(1, -1)
compare_vector_2 = user_item_matrix[movie_compare_2].values.reshape(1, -1)

# Calcular la similitud coseno
similarity_1 = cosine_similarity(target_vector, compare_vector_1)[0][0]
similarity_2 = cosine_similarity(target_vector, compare_vector_2)[0][0]

print(f"Similitud coseno entre '{movie_target}' y '{movie_compare_1}': {similarity_1}")
print(f"Similitud coseno entre '{movie_target}' y '{movie_compare_2}': {similarity_2}")

Similitud coseno entre 'One Flew Over the Cuckoo's Nest (1975)' y 'Close Shave, A (1995)': 0.23759134042122768
Similitud coseno entre 'One Flew Over the Cuckoo's Nest (1975)' y 'Antz (1998)': 0.24086135481058618


Explicación:

Los resultados de la similitud coseno muestran que "One Flew Over the Cuckoo's Nest (1975)" tiene una similitud ligeramente mayor con "Antz (1998)" (0.24) que con "Close Shave, A (1995)" (0.2376). Aunque ambas similitudes son bajas, este valor indica que las películas tienen algunas características en común según las calificaciones de los usuarios, aunque no son altamente similares.

Estos resultados se podrian mostrar de la siguiente manera
- Si un usuario disfrutó de "One Flew Over the Cuckoo's Nest (1975)", podríamos recomendarle "Antz (1998)" primero, ya que tiene una similitud coseno ligeramente mayor, y luego "Close Shave, A (1995)".

### Pregunta 4

In [16]:
def Random_Holdout(df, test_size=0.25):
    """Divide los datos asegurando que cada usuario tenga al menos una interacción en el conjunto de entrenamiento"""
    # Seleccionar al azar un porcentaje de interacciones por usuario para entrenamiento
    train_df = df.groupby('userId').apply(lambda x: x.sample(frac=1 - test_size, random_state=42)).reset_index(drop=True)
    # El resto va al conjunto de prueba
    test_df = pd.concat([df, train_df]).drop_duplicates(keep=False)

    return train_df, test_df

# Aplicar la función al dataset filtrado
train_df, test_df = Random_Holdout(filtered_data, test_size=0.3)

<ipython-input-16-b971037283ec>:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_df = df.groupby('userId').apply(lambda x: x.sample(frac=1 - test_size, random_state=42)).reset_index(drop=True)


In [17]:
# Instalar la librería scikit-surprise para algoritmos de filtrado colaborativo
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357287 sha256=9d560f3d0dac61f85d5b0c8b28898bc29816157eb3a86454e026002454df103d
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [18]:
from surprise import Dataset, Reader, KNNBasic, accuracy

# Definir el rango de calificaciones
reader = Reader(rating_scale=(1, 5))

# Cargar datos de entrenamiento
trainset = Dataset.load_from_df(train_df[['userId', 'title', 'rating']], reader).build_full_trainset()

# Crear conjunto de prueba
testset = list(test_df[['userId', 'title', 'rating']].itertuples(index=False, name=None))


In [19]:
# Configurar opciones del modelo KNN
sim_options = {
    'name': 'msd',  # Mean Squared Difference (distancia euclidiana)
    'user_based': True  # Filtrado Usuario-Usuario
}

# Crear el modelo KNN
knn = KNNBasic(k=50, sim_options=sim_options)

# Entrenar el modelo
knn.fit(trainset)


Computing the msd similarity matrix...
Done computing similarity matrix.


In [20]:
# Hacer predicciones sobre el conjunto de prueba
predictions = knn.test(testset)

In [22]:
# Calcular el RMSE
rmse = accuracy.rmse(predictions)
print(f"RMSE del modelo: {rmse:.4f}")

RMSE: 0.9236
RMSE del modelo: 0.9236


In [21]:
# Convertir predicciones a DataFrame
predictions_df = pd.DataFrame(predictions, columns=['userId', 'movieId', 'real_rating', 'predicted_rating', 'details'])
predictions_df.head()

,userId,movieId,real_rating,predicted_rating,details
0,1,Antz (1998),4,3.882777,"{'actual_k': 50, 'was_impossible': False}"
1,1,"Hunchback of Notre Dame, The (1996)",4,3.473405,"{'actual_k': 50, 'was_impossible': False}"
2,1,"Sixth Sense, The (1999)",4,4.560000,"{'actual_k': 50, 'was_impossible': False}"
3,1,Run Lola Run (Lola rennt) (1998),4,4.186618,"{'actual_k': 50, 'was_impossible': False}"
4,1,To Kill a Mockingbird (1962),4,4.590282,"{'actual_k': 50, 'was_impossible': False}"


In [23]:
# Definir umbral de relevancia
relevant_threshold = 4.0

# Inicializar contadores
hits = 0
total_relevant = 0

# Iterar sobre todas las predicciones
for _, row in predictions_df.iterrows():
    real_rating = row['real_rating']
    predicted_rating = row['predicted_rating']

    # Considerar películas con calificación real ≥ umbral como relevantes
    if real_rating >= relevant_threshold:
        total_relevant += 1
        # Si la predicción también es ≥ umbral, es un acierto
        if predicted_rating >= relevant_threshold:
            hits += 1

# Calcular el Hit Rate Global
hit_rate_global = hits / total_relevant if total_relevant > 0 else 0
print(f"Hit Rate Global: {hit_rate_global:.2f}")

Hit Rate Global: 0.47


In [24]:
# Inicializar lista para almacenar hit rates por usuario
hit_rates = []

# Agrupar predicciones por usuario
for user_id, group in predictions_df.groupby('userId'):
    hits = 0
    total_relevant = 0

    for _, row in group.iterrows():
        real_rating = row['real_rating']
        predicted_rating = row['predicted_rating']

        # Considerar películas relevantes
        if real_rating >= relevant_threshold:
            total_relevant += 1
            if predicted_rating >= relevant_threshold:
                hits += 1

    # Calcular Hit Rate para el usuario
    if total_relevant > 0:
        hit_rate_user = hits / total_relevant
        hit_rates.append(hit_rate_user)

# Calcular el Hit Rate promedio
hit_rate_avg_user = sum(hit_rates) / len(hit_rates) if len(hit_rates) > 0 else 0
print(f"Hit Rate promedio por usuario: {hit_rate_avg_user:.2f}")

Hit Rate promedio por usuario: 0.48


Interpretacion de resultados:

* Hit Rate Global (0.47): Este valor indica que, en promedio, el 47% de las recomendaciones realizadas por el sistema de recomendación basado en KNN contienen películas que los usuarios han marcado como vistas o calificadas positivamente. En otras palabras, casi la mitad de las veces, el sistema logra recomendar una película que es relevante o atractiva para los usuarios. Este es un indicador de que el sistema está funcionando de manera efectiva, aunque también hay margen para mejorar.

* Hit Rate promedio por usuario (0.48): Este valor refleja la misma métrica, pero calculada como el promedio de aciertos para cada usuario individualmente. Un 48% significa que, en promedio, cada usuario encuentra una película relevante en casi la mitad de sus recomendaciones. Este dato es útil para entender la consistencia del sistema en su capacidad de generar recomendaciones satisfactorias a nivel de cada usuario, independientemente del comportamiento o historial individual de cada uno.

Para mejorar la efectividad del sistema, se podrían explorar ajustes adicionales, como probar con un mayor número de vecinos, ajustar el filtro de similitud o incluso combinar KNN con otros métodos.